# Legolas Demo Code

Use this code to test your calibration values and as a reference for solving the challenges

In [1]:
from core import *
import utils

In [2]:
# this is how you reset the rpyc server when facing the port already in use
# need to change to actual host name
host_1 = "192.168.0.6"
host_2 = "192.168.0.7"

utils.restart_server(host=host_1)
utils.restart_server(host=host_2)

kill server at PID:2936
restarting the rpyc server at host: 192.168.0.6
rpyc server start successfully
kill server at PID:27304
restarting the rpyc server at host: 192.168.0.7
rpyc server start successfully


In [3]:
stage, depo_device, pH_device, conn1, conn2, config = load_from_config("config.yaml")

In [3]:
# it is ok if this fails, just move on to the next code block
sanity_check(stage=stage, pH_device=pH_device, depo_device=depo_device)

AssertionError: 

In [ ]:
# find the usb port if problem occur
list_ports = conn1.modules['serial.tools.list_ports']

ports=list_ports.comports()
for port, desc, hwid in sorted(ports):
    print(port, desc, hwid)

## Test Device Functionality

In [5]:
stage.home()

(195, 644)

In [4]:
# Go to Home Position
reset(stage=stage, pH_device=pH_device, depo_device=depo_device)
# stage.home()

KeyError: 'full_up'

In [5]:
# Drop pH meter and raise it up (don't run if pH path is obstructed)
pH_device.to_zpos("full_down")
pH_device.to_zpos("full_up")

In [6]:
# Drop deposition device and raise it up (don't run if syringe path is obstructed)
depo_device.to_zpos("full_down")
depo_device.to_zpos("full_up")

## Test Cell Mapping and Accuracy
Use this code to see if pH device is aligning with the cell map properly

If calibrated properly with the pH sensor, "stage" and "pH_device" will move the cart to the same spot

In [16]:
#stage.get_XYloc()

(-1134, -436)

In [7]:
stage.move_to_cell(0,0)
pH_device.to_zpos("full_down")
pH_device.to_zpos("full_up")

In [12]:
stage.move_to_cell(3, 5)
pH_device.to_zpos("full_down")
pH_device.to_zpos("full_up")

In [13]:
stage.move_to_cell(3, 0)
pH_device.to_zpos("full_down")
pH_device.to_zpos("full_up")

In [14]:
stage.move_to_cell(0, 5)
pH_device.to_zpos("full_down")
pH_device.to_zpos("full_up")

In [15]:
stage.move_to_loc("clean")
pH_device.to_zpos("full_down")
pH_device.to_zpos("full_up")

## Test pH device and Syringe Functionality

In [17]:
# this is the pH meter
pH_device

In [4]:
# the cleaning location can be used to wash off any acidic or basic solution from the probe prior to measurements
# it is not necessary to use this in your code, but if you are getting incorrect pH values you may need to clean with DI
pH_device.move_to_loc("clean")
pH_device.to_zpos("full_down")
pH_device.to_zpos("full_up")

In [22]:
# this runs the pH measuring loop, where the value passed to the function is the amount of time spent in the well
# it is suggested to measure for at least 20 seconds to let the pH measurement stabilize
pH_device.move_to_cell(row=0, col=0)
pH_device.pH_measure(20)

pH is Measured as: 3.78


3.78

In [4]:
# this is the syringe
depo_device

In [6]:
# go to acid well and test range of motion
depo_device.move_to_loc("acid")
depo_device.to_zpos("full_down")
depo_device.to_zpos("full_up")

In [5]:
# acquire 0.5 mL from the acid reservoir
depo_device.acquire(0.6, location="acid")

TimeoutError: result expired

In [27]:
# deposit the 0.5 mL into the 0,0 sample well
depo_device.deposition(0.6, row=0, col=0)

## Example Loops

Use these loops as a reference for ways to construct your autonomous experiments and the functions needed

In [28]:
depo_device.acquire(0.6, location="acid")
depo_device.deposition(0.1, row=0, col=1)
depo_device.deposition(0.2, row=0, col=2)
depo_device.deposition(0.3, row=0, col=3)

In [9]:
depo_device.to_zpos("full_up")

TimeoutError: result expired

In [6]:
for row in range(4):
    for col in range(6):
        depo_device.acquire(0.6, location="acid")
        depo_device.deposition(0.6, row=row, col=col)
        pH_device.move_to_cell(row=row, col=col)
        pH = pH_device.pH_measure(stable_time=1)
        print(f"row {row} col {col} pH {pH}")

TimeoutError: result expired

In [7]:
#pH_device.pH_positions

{'full_down': 634, 'full_up': 831}

In [6]:
#pH_device.motor_pH.get_position()

1946

In [ ]:
row = 5
for col in range(8):
    depo_device.acquire(0.6, location="acid")
    depo_device.deposition(0.6, row=row, col=col)
    pH_device.move_to_cell(row=row, col=col)
    pH = pH_device.pH_measure(stable_time=20)
    print(f"row {row} col {col} pH {pH}")